### Import necessary libraries. 

In [ ]:
# Data processing modual
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.impute import SimpleImputer

# Visualisation modual
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
%matplotlib inline

### Read data file

In [ ]:
#sets the maximum number of columns and rows displayed when a frame is pretty-printed
pd.set_option("max_rows", None)
pd.set_option("max_columns", None)

In [ ]:
d = pd.read_csv('bankruptcy.csv')
#Replace the categorical column (class) values into binary
d['class']=pd.Categorical(d['class'].replace({"b'0'" : 0, "b'1'":1}))
d.info()

In [ ]:
# Change the class variable from categorical to integer
d['class']= pd.Categorical(d['class']).codes

In [ ]:
def show_Null_plot(dataset):
    dataset.isnull().any() #check whether it is NULL for each row
    msno.matrix(dataset, labels = True, sparkline = False, figsize=(35,20), fontsize = 18)

In [ ]:
# Using opened-source to show the plot
show_Null_plot(d)

# Find top 20 columns with most null values
print(d.isnull().sum().sort_values(ascending=False).head(20))

In [ ]:
#replace the null values with 'nan'
d = d.replace('', np.nan)
nan_columns = []
nan_values = []
#count the number of  nan values in Dataset and plot a bargraph
for column in d.columns:
    nan_columns.append(column)
    nan_values.append(d[column].isnull().sum())

fig, ax = plt.subplots(figsize=(40,20))
plt.bar(nan_columns, nan_values)

Column Attr37 contains the highest number of "Nan" values and data imputation on the column will nto be useful. Remove the column and plot the bar graph again

In [ ]:
# Delete the column with highest number of nan values ie Column Attr37
d = d.drop(columns = ['Attr37'])
# Find 20 columns with most nan values
print(d.isnull().sum().sort_values(ascending=False).head(20))

show_Null_plot(d)

In [ ]:
# Plot the null values graph after removing the Attr37 column
nan_columns = []
nan_values  = []

for column in d.columns:
    nan_columns.append(column)
    nan_values.append(d[column].isnull().sum())

fig, ax = plt.subplots(figsize=(40,20))
plt.bar(nan_columns, nan_values)

## Outlier detection & Treatment
Outliers are observations that are far away from the other data points in a random sample of a population. We often want to make assumptions about a specific population. Extreme values can have a significant impact on conclusions drawn from data.

<b>Commonly used methods for detection of outliers is:</b> 
1. Tukey’s box plot method-
    Tukey distinguishes between possible and probable outliers. A possible outlier is located between the inner and the outer       fence, whereas a probable outlier is located outside the outer fence.
2. z-score method-
    For each observation (Xn), it is measured how many standard deviations the data point is away from its mean (X̄).
3. Median Absolute Deviation method-
    Replaces the mean and standard deviation with more robust statistics, like the median and median absolute deviation. The       median absolute deviation is defined as:

In [ ]:
# Method for outlier detection via Inter-quartile range method: 
def outlier_detection(x, scale):
    Q1 = x.quantile(0.25)
    Q3 = x.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - scale * IQR 
    upper = Q3 + scale * IQR  
    outliers = ((x<lower)| (x>upper)).sum()
    total_length = len(x.index)
    percent_outlier = (outliers * 100 / total_length).round(3)
    print(x.name, "\tTotal outliers is: ", outliers,"\t", "\t Percentage Outliers: ", percent_outlier )    

In [ ]:
# check number of Outliers in all columns
# Scale is 1.5 (Popular)
scale = 1.5
for column in d:
    outlier_detection(d[column], scale)

<b>For the treatement of outliers there are several methods:</b>
1. Quantile-based Flooring and Capping:
    Here points below 10th percentile are replaced with 10th percentile values
    and  points above 90th percentile are replaced with 90th percentile values.
    
2. Trimming:
      We completely remove data points that are outliers. This changes the total number of data rows present largely for our         bankruptcy data
    
3. IQR Score:
    The rule of thumb is that anything not in the range of (Q1 - 1.5 IQR) and (Q3 + 1.5 IQR) is an outlier, and can be removed
    
4. Log Transformation:
    Each cell could be transformed to a new value by formulating logarithmic, square, square root etc. But since we have           negative values too we cannot use this method
    
5. Replacing Outliers with Median Values:
    We calculate the median and aassign those values to the upper and lower datapoints

In [ ]:
# Outlier treatment (Replacing Outliers with Median Values)
df_median_treated = d.copy()
def outlier_treatment(attr,threshold):
    Q1 = attr.quantile(0.25)
    Q2 = attr.quantile(0.50)
    Q3 = attr.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - threshold * IQR 
    upper = Q3 + threshold * IQR 
    if (attr.name != 'class'):
        df_median_treated[attr.name] = np.where((df_median_treated[attr.name]>upper)|(df_median_treated[attr.name]<lower), Q2 , df_median_treated[attr.name]) 

In [ ]:
for column in df_median_treated:
    outlier_treatment(df_median_treated[column], scale)
    
df_median_treated.describe()

In [ ]:
df_median_treated.skew(axis = 0)

## Data Imputation

In [ ]:
#Data imputation via mean on outlier treated data
d_imp1 = df_median_treated.copy()
imp = SimpleImputer(missing_values = np.nan, strategy = 'mean') 
df_mean_imputed = pd.DataFrame(imp.fit_transform(d_imp1))
df_mean_imputed.rename(columns={i:d_imp1.columns[i] for i in range(d_imp1.columns.size)}, inplace = True)   
df_mean_imputed.describe().round(5) 

In [ ]:
#Data imputation via median on outlier treated data
d_imp2 = df_median_treated.copy()
imp = SimpleImputer(missing_values = np.nan, strategy = 'median') 
df_median_imputed= pd.DataFrame(imp.fit_transform(d_imp2))  # indexer method to update the dataframe
df_median_imputed.rename(columns={i:d_imp2.columns[i] for i in range(d_imp2.columns.size)}, inplace = True)
df_median_imputed.describe().round(5)       

In [ ]:
#Data imputation via mode on outlier treated data
d_imp3 = df_median_treated.copy()
imp = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent') 
df_freq_imputed = pd.DataFrame(imp.fit_transform(d_imp3))  # indexer method to update the dataframe
df_freq_imputed.rename(columns={i:d_imp3.columns[i] for i in range(d_imp3.columns.size)}, inplace = True)
df_freq_imputed.describe().round(5)          

In [ ]:
#Check Correlation matrix of mean imputed data
corr = df_mean_imputed.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(13, 13))
sns.set_style(style="white")
cmap = sns.diverging_palette(10, 250, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap,           #corr > 0.8 | corr < -0.8
            square=True, linewidths=.5, 
            cbar_kws={"shrink": .5}, 
            ax= ax , 
            xticklabels = df_mean_imputed.columns.tolist(),
            yticklabels = df_mean_imputed.columns.tolist()) 
           

In [ ]:
#Check Correlation matrix of median imputed data

corr_2 = df_median_imputed.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(15, 15))
sns.set_style(style="white")
cmap = sns.diverging_palette(10, 250, as_cmap=True)
sns.heatmap(corr>0.85, mask=mask,
            cmap=cmap,
            square=True, 
            linewidths=.5, 
            cbar_kws={"shrink": .5}, 
            ax= ax,
            xticklabels = df_median_imputed.columns.tolist(),
            yticklabels = df_median_imputed.columns.tolist())

In [ ]:
#Check Correlation matrix of mode imputed data

corr = df_freq_imputed.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(15, 15))
sns.set_style(style="white")
cmap = sns.diverging_palette(10, 250, as_cmap=True)
sns.heatmap(corr, mask=mask, 
            cmap=cmap,
            square=True, 
            linewidths=.5,
            cbar_kws={"shrink": .5},
            ax= ax,
            xticklabels = df_freq_imputed.columns.tolist(),
            yticklabels = df_freq_imputed.columns.tolist())

Create the correlation groups and finalize the variables according to the groups

In [ ]:
def correlation_group(correlation, percentile):
    correlation_pairs = correlation.unstack()
    high_correlation = correlation_pairs[(correlation_pairs>=percentile)|(correlation_pairs<= -1*percentile)]
    print(high_correlation)

In [ ]:
correlation_group(corr_2 , 0.85)

Columns with attributes 1,7,11,14,22,35,48 show positive correlation(>0.8)
Columns with attributes 3,6,10,25,38,51    show negative correlation(<-0.8)

Attributes positive correlation: 
1. Attr1 -net profit / total assets
2. Attr7 -EBIT(Earnings before interest and taxes) / total assets
3. Attr11-(gross profit + extraordinary items + financial expenses) / total assets
4. Attr14-(gross profit + interest) / total assets
5. Attr22- profit on operating activities / total assets
6. Attr35-profit on sales / total assets
7. Attr48-EBITDA (profit on operating activities - depreciation) / total assets



Attributes negative correlation:
1. Attr3 -working capital / total assets
2. Attr6 -retained earnings / total assets
3. Attr10-equity / total assets
4. Attr25-(equity - share capital) / total assets
5. Attr38-constant capital / total assets
6. Attr51-short-term liabilities / total assets


We take the above attributes in a new dataframe

In [ ]:
# Create new data frame with variables having high positive correlation
df_positive_relation = df_median_imputed[[1,7,11,14,22,35,38,48, 63]]
df_positive_relation.describe().round(3)

In [ ]:
plt.boxplot(df_positive_relation, labels =df_positive_relation.columns )
plt.show()

In [ ]:
sns.pairplot(df_positive_relation, hue= 63)

In [ ]:
# Create new data frame with variables having high negative correlation
df_negative_relation = df_median_imputed[['Attr3','Attr6','Attr10','Attr25','Attr38','Attr51']]
df_negative_relation.describe().round(3)

In [ ]:
plt.boxplot(df_negative_relation, labels = df_negative_relation.columns )
plt.show()